# Offline SMT Solvers – Lecture Notes

## Overview

In this lecture, we explore the **offline version** of SMT (Satisfiability Modulo Theories) solvers, which treat the SAT solver as a **black box**.

---

## High-Level Process

<br>

<img src="./alr-images/1110.png" width="500" style="display: block; margin: auto;">

<br>

Given a first-order formula $\varphi$, the offline SMT solver follows these steps:

1. **Boolean Abstraction**: Generate a propositional formula $\psi$ from $\varphi$.
2. **Iterative Solving Loop**:
   - Query the SAT solver with $\psi$.
   - If **unsat**, then $\varphi$ is unsatisfiable.
   - If **sat**, extract the model $A$ and compute its inverse $b^{-1}(A)$.
   - Check if $b^{-1}(A)$ is satisfiable in the theory using a theory solver.
     - If **sat**, return "SAT".
     - If **unsat**, conjoin $\neg A$ (blocking clause) with $\psi$ and repeat.

---

## Pseudocode Summary

while True:  
  result, model = SAT_Solver(ψ)  
  if result == UNSAT:  
    return UNSAT  
  if Theory_Solver(b_inverse(model)) == SAT:  
    return SAT  
  else:  
    ψ = ψ ∧ ¬model  # add blocking clause

---

## Example Recap

<br>

<img src="./alr-images/1112.png" width="500" style="display: block; margin: auto;">

<br>

Recall: Boolean abstraction ψ = b₁ ∧ b₂ ∧ b₃ ∨ ¬b₁

Only model: b₁ ∧ b₂ ∧ b₃

- Take its **negation** and **conjoin** with original ψ.
- SAT solver returns UNSAT → conclude original formula is UNSAT.

---

## Problem with Basic Blocking Clauses

<br>

<img src="./alr-images/1111.png" width="500" style="display: block; margin: auto;">

<br>

Blocking clause $\neg A$ may be **too weak**:

- Imagine $A$ has 100 literals.
- Negating $A$ blocks only that exact assignment.
- But there are $2^{98}$ other bad assignments that share key unsatisfiable parts (e.g., $x = y$ and $x < y$).

---

## Solution: Minimal Unsatisfiable Core

<br>

<img src="./alr-images/1113.png" width="500" style="display: block; margin: auto;">

<br>

Rather than blocking the whole assignment:

- Compute an **unsatisfiable core**: a subset of $A$ that is still unsatisfiable.
- Ideally, compute a **minimal** unsat core: dropping any literal would make it satisfiable.
- The smaller the core, the stronger the theory conflict clause.

### Example

Assignment includes:
- $x = y$
- $x < y$
- $a_1 \land a_2 \land$ ..., $\land a_{98}$

Unsat core: { $x = y$, $x < y$ }

This prevents all $2^{98}$ bad assignments sharing that contradiction.

---

## Simple Algorithm to Compute Minimal Unsat Core

<br>

<img src="./alr-images/1115.png" width="500" style="display: block; margin: auto;">

<br>

Given an unsatisfiable conjunction $\varphi$:

1. For each literal $\ell$ in $\varphi$:
   - Drop $\ell$, get $\varphi'$.
   - If $\varphi'$ is **still unsat**, update $\varphi := \varphi'$.
   - Else, keep $\ell$.

After iterating through all literals, $\varphi$ becomes a **minimal unsat core**.

### Example

Initial formula:
- $x = y$
- $f(x) + z = 5$
- $f(x) \ne f(y)$
- $y \le 3$

Steps:
- Drop $x = y$ → SAT → keep it.
- Drop $f(x) + z = 5$ → UNSAT → drop it.
- Drop $f(x) \ne f(y)$ → SAT → keep it.
- Drop $y \le 3$ → UNSAT → drop it.

Resulting minimal unsat core:
- $x = y$
- $f(x) \ne f(y)$

---

## Final Thoughts

<br>

<img src="./alr-images/1116.png" width="500" style="display: block; margin: auto;">

<br>

To improve efficiency in offline SMT solvers:

- When SAT solver returns a model, check it using the theory solver.
- If unsatisfiable, **compute and negate a minimal unsat core**, not the whole model.
- Conjoin this **stronger conflict clause** with $\psi$.

### Benefits:
- No need to convert to DNF.
- Still treats SAT solver as a black box.

---

In the next lecture, we’ll explore a **more efficient** approach by **opening the SAT solver black box**.
